In [1]:
# Optimization - searching for the optimal value
# MaxCut : NP-Complete
# Map onto QC manually
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram

In [ ]:
# Generate Graph


In [7]:
# Computing the weight matrix from the random graph
w = np.zeros([6, 6])
# w = [[0,0,1,1,0,0],
#     [0,0,1,1,0,0],
#     [1,1,0,0,1,1],
#    [1,1,0,0,1,1],
#     [0,0,1,1,0,0],
#    [0,0,1,1,0,0]]
w[0,2] = w[0,3] = 1
w[1,2] = w[1,3] = 1
w[2,0] = w[2,1] = w[2,4] = w[2,5] = 1
w[3,0] = w[3,1] = w[3,4] = w[3,5] = 1
w[4,2] = w[4,3] = 1
w[5,2] = w[5,3] = 1
print(w)

[[0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1.]
 [1. 1. 0. 0. 1. 1.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0.]]


In [ ]:
# Brute force, 2^n combinations, 2**6 = 64
# expected answer is 8 edges for this graph

In [8]:
# Map to the Ising Problem - Create Ising Hamiltonian
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Max-cut

Maximize
 obj: 2 x_0 + 2 x_1 + 4 x_2 + 4 x_3 + 2 x_4 + 2 x_5 + [ - 4 x_0*x_2 - 4 x_0*x_3
      - 4 x_1*x_2 - 4 x_1*x_3 - 4 x_2*x_4 - 4 x_2*x_5 - 4 x_3*x_4 - 4 x_3*x_5
      ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
End



In [9]:
qubitOp, offset = qp.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

Offset: -4.0
Ising Hamiltonian:
0.5 * ZIZIII
+ 0.5 * IZZIII
+ 0.5 * ZIIZII
+ 0.5 * IZIZII
+ 0.5 * IIZIZI
+ 0.5 * IIIZZI
+ 0.5 * IIZIIZ
+ 0.5 * IIIZIZ


In [10]:
# solving Quadratic Program using exact classical eigensolver
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact.solve(qp)
print(result)

optimal function value: 8.0
optimal value: [1. 1. 0. 0. 1. 1.]
status: SUCCESS


In [11]:
# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)

x = max_cut.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

energy: -4.0
max-cut objective: -8.0
solution: [1 1 0 0 1 1]
solution objective: 8.0


In [12]:
# Running it on a QC
algorithm_globals.random_seed = 123
seed = 10598
backend = Aer.get_backend("aer_simulator_statevector")
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

In [13]:
# Construct VQE
spsa = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
vqe = VQE(ry, optimizer=spsa, quantum_instance=quantum_instance)

# run VQE
result = vqe.compute_minimum_eigenvalue(qubitOp)

# print results
x = max_cut.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

energy: -3.976425383546248
time: 3.294271945953369
max-cut objective: -7.976425383546248
solution: [1. 1. 0. 0. 1. 1.]
solution objective: 8.0


In [14]:
# create minimum eigen optimizer based on VQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result)

optimal function value: 8.0
optimal value: [0. 0. 1. 1. 0. 0.]
status: SUCCESS
